# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer.csv", encoding="UTF-8")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,"大野 あや子,1",女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,"宇多田 貴美子,1",女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

### はじめに：パイプ演算子について  
パイプ演算子 %>% はRの標準演算子ではなく、dplyrをインストールしたら使えるようになります。  
https://qiita.com/zakkiiii/items/ecb7ddf55c7f1bfacecf  

R-4.1.0 からはパイプ演算子がRの標準装備になるようです。  
The native pipe  
When the {magrittr} package introduced the pipe to R in 2014, it was building upon similar syntax in Unix / linux scripting languages (bash) and functional programming languages (like F#).  
https://www.r-bloggers.com/2021/05/new-features-in-r-4-1-0/

---
> R-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
# これが実行できない
# inner_join(df_receipt, df_store[c('store_cd', 'store_name')], by = 'store_cd') %>% ３６slice(1:10)

In [8]:
# df_tmp <- inner_join(df_receipt, df_store[c('store_cd', 'store_name')], by = 'store_cd')# %>% slice(1:10)
# head(df_tmp, n=10)

---
> R-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [44]:
# inner_join(df_product,
#            df_category[c('category_major_cd', 'category_medium_cd','category_small_cd','category_small_name')],
#            by = c('category_major_cd', 'category_medium_cd','category_small_cd')) %>%
#     slice(1:10)

---
> R-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [8]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount))

df_target <- df_customer %>%
    filter(!grepl("^Z", customer_id) & gender_cd == '1')

left_join(df_target['customer_id'], df_sum, by = 'customer_id') %>%
    replace_na(list(sum_amount=0)) %>%
    slice(1:10)

`summarise()` ungrouping output (override with `.groups` argument)


customer_id,sum_amount
CS001215000145,875
CS033513000180,868
CS011215000048,3444
CS009413000079,0
CS040412000191,210
CS046615000013,0
CS002613000047,0
CS023513000066,771
CS011312000159,0
CS004613000404,0


---
> R-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [9]:
df_sum <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(sum_amount=sum(amount)) %>%
    arrange(desc(sum_amount)) %>%
    slice(1:20)

df_cnt <- df_receipt %>%
    filter(!grepl("^Z", customer_id)) %>%
    group_by(customer_id) %>%
    summarise(come_days=n_distinct(sales_ymd)) %>%
    arrange(desc(come_days), customer_id) %>%
    slice(1:20)

full_join(df_sum, df_cnt, by = "customer_id")

`summarise()` ungrouping output (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)


customer_id,sum_amount,come_days
CS017415000097,23086,20
CS015415000185,20153,22
CS031414000051,19202,19
CS028415000007,19127,21
CS001605000009,18925,NA
CS010214000010,18585,22
CS006515000023,18372,NA
CS016415000141,18372,20
CS011414000106,18338,NA
CS038415000104,17847,NA


---
> R-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [47]:
df_store_tmp <- df_store
df_product_tmp <- df_product
df_store_tmp['key'] <- 0
df_product_tmp['key'] <- 0
nrow(full_join(df_store_tmp, df_product_tmp, by = "key"))

[1] 531590